In [92]:
!pip install pmdarima
!pip3 install numpy scipy patsy pandas
!pip3 install statsmodels

In [93]:
import pandas as pd 
import numpy as np
import datetime
from datetime import datetime
import _strptime
import altair as alt

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.naive_bayes import GaussianNB
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten

# Pregunta numero 1

En la mayoria de las variables podemos observar una distribucion relativamente uniforme de los datos. A continuacion podemos encontrar las variables que poseen una tendencia a acumularse en cierto punto dentro de su distribucion.

In [142]:
df1 = pd.read_csv('/content/dataset_SCL.csv')

origen = df1['TIPOVUELO'].value_counts().reset_index()
origen = pd.DataFrame(origen)
origen.rename(columns = {'index':'tipo'}, inplace = True)
origen.rename(columns = {'TIPOVUELO':'Cantidad'}, inplace = True)
origen['Porcentaje'] = origen['Cantidad']/origen['Cantidad'].sum() 
origen = origen.sort_values('Porcentaje',ascending= False)
display(origen.head(2))

destino = df1['SIGLADES'].value_counts().reset_index()
destino = pd.DataFrame(destino)
destino.rename(columns = {'index':'Destino'}, inplace = True)
destino.rename(columns = {'SIGLADES':'Cantidad'}, inplace = True)
destino['Porcentaje'] = destino['Cantidad']/destino['Cantidad'].sum() 
destino = destino.sort_values('Porcentaje',ascending= False)
display(destino.head(2))

opera = df1['OPERA'].value_counts().reset_index()
opera = pd.DataFrame(opera)
opera.rename(columns = {'index':'Opera'}, inplace = True)
opera.rename(columns = {'OPERA':'Cantidad'}, inplace = True)
opera['Porcentaje'] = opera['Cantidad']/opera['Cantidad'].sum() 
opera = opera.sort_values('Porcentaje',ascending= False)
display(opera.head(4))  

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,tipo,Cantidad,Porcentaje
0,N,36966,0.541976
1,I,31240,0.458024


,Destino,Cantidad,Porcentaje
0,Buenos Aires,6335,0.092880
1,Antofagasta,5786,0.084831


,Opera,Cantidad,Porcentaje
0,Grupo LATAM,40892,0.599537
1,Sky Airline,14298,0.209630
2,Aerolineas Argentinas,1949,0.028575
3,Copa Air,1850,0.027124


Podemos observar que la tendencia a los vuelos internacionales es mayor a la de los nacionales, tambien es necesario resaltar que los vuelos a los aeropuertos cercanos a los de santiago son los mas comunes. Finalmente, es importante ver que aunque la mayoria de los vuelos son llevados a cabo por LATAM, SKY Airline tambien tiene una participacion significativa dentro de la base de datos, creando de esta forma una distribucion un poco mas uniforme dentro del dataset.

# Pregunta numero 2

In [94]:
df = pd.read_csv('/content/dataset_SCL.csv')
df = df.drop(columns=['DIA','MES'])
df['Fecha-I'] = df['Fecha-I'].astype(str)
df['DIA'] = df['Fecha-I'].str[8:10]
df['MES'] = df['Fecha-I'].str[5:7]
df[['MES','DIA']] = df[['MES','DIA']].astype(int)
df['Fecha-I'] = pd.to_datetime(df['Fecha-I'])
df['Fecha-O'] = pd.to_datetime(df['Fecha-O'])
#------------------------------------------------------------------------------------------------------------------------------------------
                                            # 2.A
#------------------------------------------------------------------------------------------------------------------------------------------

conditions = [(df['MES'] <= 2),             
             ((df['MES'] == 3) & (df['DIA'] <= 3)),              
             ((df['MES'] == 7) & (df['DIA'] >= 15)),              
             ((df['MES'] == 9) & (df['DIA'] >= 11)),
             ((df['MES'] == 12) & (df['DIA'] >= 15))]

#Resultado en base a las condiciones
choices = ['1', '1', '1', '1', '1']
df['temporada_alta'] = np.select(conditions, choices, default='0')


#------------------------------------------------------------------------------------------------------------------------------------------
                                            # 2.B
#------------------------------------------------------------------------------------------------------------------------------------------

df['dif_1'] = df['Fecha-O'] - df['Fecha-I']
df['dif_1'] = df['dif_1'].astype(str)
new = df["dif_1"].str.split(" ", n = 1, expand = True)
df["Dia"]= new[0]
df["Hora"]= new[1]
df["Dia"] = df["Dia"].str.replace('-1','1')
df["Hora"] = df["Hora"].str.replace('days','')
df["Hora"] = df["Hora"].str.replace('+','')
df["Hora"] = df["Hora"].str.replace(' ','')
new = df["Hora"].str.split(":", n = 2, expand = True)
df["Hora"]= new[1]
df["Hora"] = df["Hora"].astype(int)
df["Dia"] = df["Dia"].astype(int)

lista = []
for index,row in df.iterrows():
  dia = row.get('Dia')
  hora = row.get('Hora')
  if dia==0:
    lista.append(hora)
  if dia==1:
    lista.append(-60+hora)
df['dif_min'] = lista
df['dif_min'] = df['dif_min'].astype(int)
df = df.drop(columns=['dif_1','Dia','Hora'])

#------------------------------------------------------------------------------------------------------------------------------------------
                                            # 2.C
#------------------------------------------------------------------------------------------------------------------------------------------

conditions = [(df['dif_min'] > 15),             
              (df['dif_min'] <= 15)]
#Resultado en base a las condiciones
choices = ['1','0']
df['atraso_15'] = np.select(conditions, choices, default='0')

#------------------------------------------------------------------------------------------------------------------------------------------
                                            # 2.D
#------------------------------------------------------------------------------------------------------------------------------------------

df["Fecha-I"] = df["Fecha-I"].astype(str)
new = df["Fecha-I"].str.split(" ", n = 1, expand = True)
df["Hora"]= new[1]
df["Hora"] = df["Hora"].astype(str)
df["Hora"] = df["Hora"].str[0:2]
df["Hora"] = df["Hora"].astype(int)

conditions = [(df['Hora'] >= 5) & (df['Hora'] < 12),             
              (df['Hora'] >= 12) & (df['Hora'] < 19),
              (df['Hora'] >= 19) & (df['Hora'] < 24),
              (df['Hora'] < 5)]
#Resultado en base a las condiciones
choices = ['Mañana','Tarde','Noche','Noche']
df['periodo_dia'] = np.select(conditions, choices, default='0')
df = df.drop(columns=['Hora'])
df['AÑO'].unique()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


array([2017, 2018])

# Pregunta numero 3

In [95]:
alt.data_transformers.enable('default', max_rows=None)

df['dif_min'] = df['dif_min'].astype(int)
df_periodo_dia = df[['periodo_dia','MES','dif_min']]
df_periodo_dia = df_periodo_dia.groupby(['periodo_dia','MES']).sum().reset_index()

grafico1 = alt.Chart(df_periodo_dia).mark_bar().encode(
    x=alt.X("MES", title = ( "MES")),
    y=alt.Y('dif_min', title = ( "dif_min")),
    tooltip = "dif_min"
).properties(title= "Diferencia minutos",width=350)
grafico1
alt.layer(grafico1).facet(
    'periodo_dia',title = "Periodo"
).properties(title= "Atrasos")

alt.FacetChart(...)

##### Grafico estacionalidad

In [96]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# #Se define el vector objetivo
# variable = 'dif_min'

# #Se definen las dimensiones del gráfico
# fig, ax = plt.subplots(figsize=(15, 6))

# #Se define el tipo de gráfico, en el que el eje "x" representa mes y el eje "y" el vector objetivo
# #esta informacion se agrupa por año
# sns.lineplot(df['Fecha-I'], df[variable], hue=df['MES'], err_style = None)
# ax.set_title('Gráfico de estacionalidad', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
# ax.set_xlabel('MES', fontsize = 16, fontdict=dict(weight='bold'))
# ax.set_ylabel('Contrataciones', fontsize = 16, fontdict=dict(weight='bold'))

# fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

# sns.boxplot(df['AÑO'], df[variable], ax=ax[0])
# ax[0].set_title('Año como Box Plot\n(La tendencia)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
# ax[0].set_xlabel('AÑO', fontsize = 16, fontdict=dict(weight='bold'))
# ax[0].set_ylabel('contrataciones', fontsize = 16, fontdict=dict(weight='bold'))

# sns.boxplot(df['MES'], df[variable], ax=ax[1])
# ax[1].set_title('Mes como Box Plot\n(Estacionalidad)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
# ax[1].set_xlabel('MES', fontsize = 16, fontdict=dict(weight='bold'))
# ax[1].set_ylabel('Contrataciones', fontsize = 16, fontdict=dict(weight='bold'))

### Composicion tasa de atraso

##### Tasa de atraso por destino programado

In [97]:
atraso_destino = df[df['dif_min']>0]
atraso_destino = atraso_destino['Des-I'].value_counts().reset_index()
atraso_destino = pd.DataFrame(atraso_destino)
atraso_destino.rename(columns = {'index':'Destino'}, inplace = True)
atraso_destino.rename(columns = {'Des-I':'Cantidad'}, inplace = True)
atraso_destino['Porcentaje'] = atraso_destino['Cantidad']/atraso_destino['Cantidad'].sum() 
atraso_destino = atraso_destino.sort_values('Porcentaje',ascending= False)
atraso_destino

,Destino,Cantidad,Porcentaje
0,SPJC,3675,0.080919
1,SCFA,3513,0.077352
2,SCTE,3179,0.069997
3,SCCF,2985,0.065726
4,SCDA,2629,0.057887
...,...,...,...
58,SBFI,1,0.000022
59,SEQU,1,0.000022
60,SEQM,1,0.000022
61,SBCT,1,0.000022


##### Tasa de atraso por Aerolinea de vuelo programado

In [98]:
atraso_aerolinea = df[df['dif_min']>0]
atraso_aerolinea = atraso_aerolinea['Emp-I'].value_counts().reset_index()
atraso_aerolinea = pd.DataFrame(atraso_aerolinea)
atraso_aerolinea.rename(columns = {'index':'Aerolinea'}, inplace = True)
atraso_aerolinea.rename(columns = {'Emp-I':'Cantidad'}, inplace = True)
atraso_aerolinea['Porcentaje'] = atraso_aerolinea['Cantidad']/atraso_aerolinea['Cantidad'].sum() 
atraso_aerolinea = atraso_aerolinea.sort_values('Porcentaje',ascending= False)
atraso_aerolinea.head(10)

,Aerolinea,Cantidad,Porcentaje
0,LAN,27501,0.605535
1,SKU,8415,0.185287
2,TAM,2393,0.052691
3,LAW,1165,0.025652
4,ARG,1150,0.025321
5,CMP,684,0.015061
6,AVA,658,0.014488
7,JAT,647,0.014246
8,GLO,471,0.010371
9,ACA,433,0.009534


##### Tasa de atraso por mes del año

In [99]:
atraso_por_mes = df[df['dif_min']>0]
atraso_por_mes = atraso_por_mes['MES'].value_counts().reset_index()
atraso_por_mes = pd.DataFrame(atraso_por_mes)
atraso_por_mes.rename(columns = {'index':'Mes'}, inplace = True)
atraso_por_mes.rename(columns = {'MES':'Cantidad'}, inplace = True)
atraso_por_mes['Porcentaje'] = atraso_por_mes['Cantidad']/atraso_por_mes['Cantidad'].sum() 
atraso_por_mes = atraso_por_mes.sort_values('Porcentaje',ascending= False)
atraso_por_mes.head(12)

,Mes,Cantidad,Porcentaje
0,12,4703,0.103554
1,7,4592,0.101110
2,10,4290,0.094460
3,11,4205,0.092589
4,1,4037,0.088889
5,8,3707,0.081623
6,2,3625,0.079818
7,9,3619,0.079686
8,5,3348,0.073719
9,6,3225,0.071010


##### Tasa de atraso por dia de la semana

In [100]:
atraso_por_mes = df[df['dif_min']>0]
atraso_por_mes = atraso_por_mes['DIANOM'].value_counts().reset_index()
atraso_por_mes = pd.DataFrame(atraso_por_mes)
atraso_por_mes.rename(columns = {'index':'Dia'}, inplace = True)
atraso_por_mes.rename(columns = {'DIANOM':'Cantidad'}, inplace = True)
atraso_por_mes['Porcentaje'] = atraso_por_mes['Cantidad']/atraso_por_mes['Cantidad'].sum() 
atraso_por_mes = atraso_por_mes.sort_values('Porcentaje',ascending= False)
atraso_por_mes.head(12)

,Dia,Cantidad,Porcentaje
0,Viernes,7367,0.162212
1,Jueves,6991,0.153933
2,Lunes,6825,0.150277
3,Miercoles,6406,0.141052
4,Domingo,6250,0.137617
5,Martes,6193,0.136362
6,Sabado,5384,0.118549


##### Tasa de atraso por temporada


In [101]:
atraso_por_temporada = df[df['dif_min']>0]
atraso_por_temporada = atraso_por_temporada['temporada_alta'].value_counts().reset_index()
atraso_por_temporada = pd.DataFrame(atraso_por_temporada)
atraso_por_temporada.rename(columns = {'index':'Temporada'}, inplace = True)
atraso_por_temporada.rename(columns = {'temporada_alta':'Cantidad'}, inplace = True)
atraso_por_temporada['Porcentaje'] = atraso_por_temporada['Cantidad']/atraso_por_temporada['Cantidad'].sum() 
atraso_por_temporada = atraso_por_temporada.sort_values('Porcentaje',ascending= False)
atraso_por_temporada["Temporada"] = atraso_por_temporada["Temporada"].str.replace('0','Baja')
atraso_por_temporada["Temporada"] = atraso_por_temporada["Temporada"].str.replace('1','Alta')
atraso_por_temporada

,Temporada,Cantidad,Porcentaje
0,Baja,29887,0.658072
1,Alta,15529,0.341928


##### Tasa de atraso por tipo de vuelo

In [102]:
atraso_por_tipo_vuelo = df[df['dif_min']>0]
atraso_por_tipo_vuelo = atraso_por_tipo_vuelo['TIPOVUELO'].value_counts().reset_index()
atraso_por_tipo_vuelo = pd.DataFrame(atraso_por_tipo_vuelo)
atraso_por_tipo_vuelo.rename(columns = {'index':'Tipo_Vuelo'}, inplace = True)
atraso_por_tipo_vuelo.rename(columns = {'TIPOVUELO':'Cantidad'}, inplace = True)
atraso_por_tipo_vuelo['Porcentaje'] = atraso_por_tipo_vuelo['Cantidad']/atraso_por_tipo_vuelo['Cantidad'].sum() 
atraso_por_tipo_vuelo = atraso_por_tipo_vuelo.sort_values('Porcentaje',ascending= False)
atraso_por_tipo_vuelo["Tipo_Vuelo"] = atraso_por_tipo_vuelo["Tipo_Vuelo"].str.replace('N','Nacional')
atraso_por_tipo_vuelo["Tipo_Vuelo"] = atraso_por_tipo_vuelo["Tipo_Vuelo"].str.replace('I','Internacional')
atraso_por_tipo_vuelo

,Tipo_Vuelo,Cantidad,Porcentaje
0,Nacional,24276,0.534525
1,Internacional,21140,0.465475


#### Variables extras que podrian influir

In [103]:
display(df.columns)
# Ciudad de orgien programado Ori-I
# Ciudad de origen de operacion Ori-O que va de la mano con la ciudad de origen SIGLADES
df

Index(['Fecha-I', 'Vlo-I', 'Ori-I', 'Des-I', 'Emp-I', 'Fecha-O', 'Vlo-O',
       'Ori-O', 'Des-O', 'Emp-O', 'AÑO', 'DIANOM', 'TIPOVUELO', 'OPERA',
       'SIGLAORI', 'SIGLADES', 'DIA', 'MES', 'temporada_alta', 'dif_min',
       'atraso_15', 'periodo_dia'],
      dtype='object')

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,DIA,MES,temporada_alta,dif_min,atraso_15,periodo_dia
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,I,American Airlines,Santiago,Miami,1,1,1,3,0,Noche
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,I,American Airlines,Santiago,Miami,2,1,1,9,0,Noche
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,I,American Airlines,Santiago,Miami,3,1,1,9,0,Noche
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,I,American Airlines,Santiago,Miami,4,1,1,3,0,Noche
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,I,American Airlines,Santiago,Miami,5,1,1,-2,0,Noche
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,I,JetSmart SPA,Santiago,Lima,22,12,1,46,1,Tarde
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,I,JetSmart SPA,Santiago,Lima,25,12,1,16,1,Tarde
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,I,JetSmart SPA,Santiago,Lima,27,12,1,40,1,Tarde
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,I,JetSmart SPA,Santiago,Lima,29,12,1,13,0,Tarde


# Pregunta numero 4

Vamos a dividir el dataset en temporada alta y baja para poder estudiar las variables por separado, ya que, tienen una gran diferencia en cantidad de atrasos

In [144]:
print(len(df[(df['temporada_alta']=='1') & (df['dif_min']>0)]))
print(len(df[(df['temporada_alta']=='0') & (df['dif_min']>0)]))

15529
29887


In [145]:
df_alta = df[df['temporada_alta']=='1']
df_baja = df[df['temporada_alta']=='0']

### Temporada alta

In [143]:
df_alta.head(2)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,DIA,MES,temporada_alta,dif_min,atraso_15,periodo_dia
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,I,American Airlines,Santiago,Miami,1,1,1,3,0,Noche
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,I,American Airlines,Santiago,Miami,2,1,1,9,0,Noche


#### Gaussian NB

In [148]:
df_gaussian = df_alta.copy()
df_gaussian.columns
df_gaussian['dif_min'] = df_gaussian['dif_min'].astype(int)
lista_atraso = []
for index,row in df_gaussian.iterrows():
  min = row.get('dif_min')
  if min>=1:
    lista_atraso.append('1')
  else:lista_atraso.append('0')
df_gaussian['dummy_atraso'] = lista_atraso
df_gaussian = df_gaussian.drop(columns=['Fecha-I','Fecha-O','DIA','MES','AÑO','Vlo-I','atraso_15','SIGLAORI','OPERA'])
df_gaussian["periodo_dia"] = df_gaussian["periodo_dia"].str.replace('Noche','1')
df_gaussian["periodo_dia"] = df_gaussian["periodo_dia"].str.replace('Tarde','2')
df_gaussian["periodo_dia"] = df_gaussian["periodo_dia"].str.replace('Mañana','3')
df_gaussian["TIPOVUELO"] = df_gaussian["TIPOVUELO"].str.replace('I','1')
df_gaussian["TIPOVUELO"] = df_gaussian["TIPOVUELO"].str.replace('N','2')
df_gaussian = df_gaussian[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']]
df_gaussian[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']] = df_gaussian[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']].astype(int)
display(df_gaussian.corr())

X = df_gaussian.drop('dummy_atraso',axis=1)
y = df_gaussian['dummy_atraso']
predictors = X
targets = y
X_train, X_test, y_train, y_test  =   train_test_split(predictors, targets, test_size=.8)
print( "Predictor - Training : ", X_train.shape, "Predictor - Testing : ", X_test.shape )

classifier=GaussianNB()
classifier=classifier.fit(X_train,y_train)
predictions=classifier.predict(X_test)
a =  sklearn.metrics.confusion_matrix(y_test,predictions)
print('')
print('')
print('')
print('Falsos correctamente predecidos     = '+str("{:.2f}".format((a[0][0]/(a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')
print('')
print('Verdaderos correctamente predecidos = '+str("{:.2f}".format((a[1][1]/(a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')
print('')
print('Error                               = '+str("{:.2f}".format((a[0][1]+a[1][0])/((a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')

,TIPOVUELO,temporada_alta,dif_min,periodo_dia,dummy_atraso
TIPOVUELO,1.000000,NaN,-0.091617,0.159937,-0.025689
temporada_alta,NaN,NaN,NaN,NaN,NaN
dif_min,-0.091617,NaN,1.000000,-0.045685,0.583102
periodo_dia,0.159937,NaN,-0.045685,1.000000,-0.006204
dummy_atraso,-0.025689,NaN,0.583102,-0.006204,1.000000


Predictor - Training :  (4552, 4) Predictor - Testing :  (18210, 4)



Falsos correctamente predecidos     = 30.15%

Verdaderos correctamente predecidos = 62.59%

Error                               = 7.26%


#### SVM

In [149]:
df_svm = df_alta.copy()
df_svm.columns
df_svm['dif_min'] = df_svm['dif_min'].astype(int)
lista_atraso = []
for index,row in df_svm.iterrows():
  min = row.get('dif_min')
  if min>=1:
    lista_atraso.append('1')
  else:lista_atraso.append('0')
df_svm['dummy_atraso'] = lista_atraso
df_svm = df_svm.drop(columns=['Fecha-I','Fecha-O','DIA','MES','AÑO','Vlo-I','atraso_15','SIGLAORI','OPERA'])
df_svm["periodo_dia"] = df_svm["periodo_dia"].str.replace('Noche','1')
df_svm["periodo_dia"] = df_svm["periodo_dia"].str.replace('Tarde','2')
df_svm["periodo_dia"] = df_svm["periodo_dia"].str.replace('Mañana','3')

df_svm["TIPOVUELO"] = df_svm["TIPOVUELO"].str.replace('I','1')
df_svm["TIPOVUELO"] = df_svm["TIPOVUELO"].str.replace('N','2')
df_svm = df_svm[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']]
df_svm[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']] = df_svm[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']].astype(int)
display(df_svm.corr())

X = df_svm.drop('dummy_atraso',axis=1)
y = df_svm['dummy_atraso']


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.7,random_state=2)

Scaler_X = StandardScaler()
X_train = Scaler_X.fit_transform(X_train)
X_test = Scaler_X.transform(X_test)

rfc = RandomForestClassifier(n_estimators=1)
rfc.fit(X_train,y_train)
rfc__pred = rfc.predict(X_test)
# print(accuracy_score(y_test,rfc__pred))
a = confusion_matrix(y_test,rfc__pred)
# print(classification_report(y_test,rfc__pred))
print('')
print('')
print('')
print('Falsos correctamente predecidos     = '+str("{:.2f}".format((a[0][0]/(a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')
print('')
print('Verdaderos correctamente predecidos = '+str("{:.2f}".format((a[1][1]/(a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')
print('')
print('Error                               = '+str("{:.2f}".format((a[0][1]+a[1][0])/((a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,TIPOVUELO,temporada_alta,dif_min,periodo_dia,dummy_atraso
TIPOVUELO,1.000000,NaN,-0.091617,0.159937,-0.025689
temporada_alta,NaN,NaN,NaN,NaN,NaN
dif_min,-0.091617,NaN,1.000000,-0.045685,0.583102
periodo_dia,0.159937,NaN,-0.045685,1.000000,-0.006204
dummy_atraso,-0.025689,NaN,0.583102,-0.006204,1.000000





Falsos correctamente predecidos     = 32.20%

Verdaderos correctamente predecidos = 67.80%

Error                               = 0.00%


#### Redes Neuronales (para predecir los minutos que vamos a tener de atraso)

In [150]:
df_rn = df_alta.copy()
df_rn.columns
df_rn['dif_min'] = df_rn['dif_min'].astype(int)
lista_atraso = []
for index,row in df_rn.iterrows():
  min = row.get('dif_min')
  if min>=1:
    lista_atraso.append('1')
  else:lista_atraso.append('0')
df_rn['dummy_atraso'] = lista_atraso
df_rn = df_rn.drop(columns=['Fecha-O','DIA','MES','AÑO','Vlo-I','atraso_15','SIGLAORI','OPERA'])
df_rn["periodo_dia"] = df_rn["periodo_dia"].str.replace('Noche','1')
df_rn["periodo_dia"] = df_rn["periodo_dia"].str.replace('Tarde','2')
df_rn["periodo_dia"] = df_rn["periodo_dia"].str.replace('Mañana','3')

df_rn["TIPOVUELO"] = df_rn["TIPOVUELO"].str.replace('I','1')
df_rn["TIPOVUELO"] = df_rn["TIPOVUELO"].str.replace('N','2')
df_rn = df_rn[['Fecha-I','dif_min']]
df_rn['dif_min'] = df_rn['dif_min'].astype(int)
df_rn = df_rn[df_rn['dif_min']>0]
df_rn



# # Hacemos la data de prediccion y la que vamos a predecir
atraso1 = df_rn.iloc[15509:]
atraso2 = df_rn.iloc[:15509]

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn import pipeline


PASOS = 20
# convert series to supervised learning
def series_to_supervised(atraso2, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(atraso2) is list else atraso2.shape[1]
    df = pd.DataFrame(atraso2)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# load dataset
values = atraso2["dif_min"].values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, PASOS, 1)

values = reframed.values
n_train_days = 148 - (30+PASOS)
train = values[:n_train_days, :]
test = values[n_train_days:, :]
# split into input and outputs
x_train, y_train = train[:, :-1], train[:, -1]
x_val, y_val = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))



def crear_modeloFF():
    model = Sequential() 
    model.add(Dense(PASOS, input_shape=(1,PASOS),activation='tanh'))
    model.add(Flatten())
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_absolute_error',optimizer='Adam',metrics=["mse"])
    model.summary()
    return model

EPOCHS=40
 
model = crear_modeloFF()
 
history=model.fit(x_train,y_train,epochs=EPOCHS,validation_data=(x_val,y_val),batch_size=PASOS)


ultimosDias = atraso2.tail(30)


values = ultimosDias["dif_min"].values
values = values.astype('float32')
# normalize features
values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, PASOS, 1)
reframed.drop(reframed.columns[[7]], axis=1, inplace=True)
reframed.head()


values = reframed.values
x_test = values[6:, :]
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))


def agregarNuevoValor(x_test,nuevoValor):
    for i in range(x_test.shape[2]-1):
        x_test[0][0][i] = x_test[0][0][i+1]
    x_test[0][0][x_test.shape[2]-1]=nuevoValor
    return x_test
 
results=[]
for i in range(20):
    parcial=model.predict(x_test)
    results.append(parcial[0])
    x_test=agregarNuevoValor(x_test,parcial[0])

adimen = [x for x in results] 
inverted = scaler.inverse_transform(adimen)


prediccion1SemanaDiciembre = pd.DataFrame(inverted)
prediccion1SemanaDiciembre.columns = ['pronostico']
# prediccion1SemanaDiciembre.plot()
print('Error = ' +str("{:.2f}".format((atraso1['dif_min'].sum()-prediccion1SemanaDiciembre['pronostico'].sum())/(atraso1['dif_min'].sum())*100))+'%')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1, 20)             420       
                                                                 
 flatten_3 (Flatten)         (None, 20)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
5/5 [==============================] - 2s 262ms/step - loss: 0.5431 - mse: 0.4904 - val_loss: 0.3878 - val_mse: 0.2747
Epoch 2/40
5/5 [==============================] - 1s 323ms/step - loss: 0.5110 - mse: 0.4496 - val_loss: 0.3521 - val_mse: 0.2457
Epoch 3/40
5/5 [==============================] - 1s 323ms/step - loss: 0.4920 - mse: 0.4224 - val_loss: 0.3350 - val_mse: 0.2345
Epoch 4/40
5/5 [==============================] - 3s 650ms/step - loss: 0.4784 - mse: 0.4050 - val_loss: 0.3276 - val_mse: 0.2305
Epoch 5/40
5/5 [==============================] - 3s 647ms/step - loss: 0.4680 - mse: 0.3896 - val_loss: 0.3237 - val_mse: 0.2282
Epoch 6/40
5/5 [==============================] - 1s 326ms/step - loss: 0.4571 - mse: 0.3760 - val_loss: 0.3212 - val_mse: 0.2256
Epoch 7/40
5/5 [==============================] - 3s 649ms/step - loss: 0.4485 - mse: 0.36

Podemos observar que el modelo de Gaussian NB presentado en primer lugar nos entrega una precision muy alta sobre la probabilidad de que un vuelo se atrase o no, llegando a una precision cercana a un 90% en el entrenamiento del modelo. En segundo lugar, tenemos el modelo de SVM, el cual, es un modelo muy poderoso a nivel de machine learning pero que lamentablemente tiene problemas de overfitting y es por eso que se decidio dejarlo de lado. Finalmente, tenemos el modelo de redes neuronales que nos entrega un error cercano al 50%.
Porducto de lo anterior, para la temporada alta se utilizara el modelo de Gaussian NB, el cual trabaja en base a variables aleatorias y arboles de decision para poder clasificar las diferentes variables.

### Temporada baja

In [151]:
df_baja.head(2)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,DIA,MES,temporada_alta,dif_min,atraso_15,periodo_dia
11670,2017-03-04 23:30:00,226,SCEL,KMIA,AAL,2017-03-04 23:20:00,226,SCEL,KMIA,AAL,...,I,American Airlines,Santiago,Miami,4,3,0,-10,0,Noche
11671,2017-03-06 23:00:00,226,SCEL,KMIA,AAL,2017-03-06 22:55:00,226,SCEL,KMIA,AAL,...,I,American Airlines,Santiago,Miami,6,3,0,-5,0,Noche


#### Gaussian NB

In [156]:
df_gaussian = df_baja.copy()
df_gaussian.columns
df_gaussian['dif_min'] = df_gaussian['dif_min'].astype(int)
lista_atraso = []
for index,row in df_gaussian.iterrows():
  min = row.get('dif_min')
  if min>=1:
    lista_atraso.append('1')
  else:lista_atraso.append('0')
df_gaussian['dummy_atraso'] = lista_atraso
df_gaussian = df_gaussian.drop(columns=['Fecha-I','Fecha-O','DIA','MES','AÑO','Vlo-I','atraso_15','SIGLAORI','OPERA'])
df_gaussian["periodo_dia"] = df_gaussian["periodo_dia"].str.replace('Noche','1')
df_gaussian["periodo_dia"] = df_gaussian["periodo_dia"].str.replace('Tarde','2')
df_gaussian["periodo_dia"] = df_gaussian["periodo_dia"].str.replace('Mañana','3')
df_gaussian["TIPOVUELO"] = df_gaussian["TIPOVUELO"].str.replace('I','1')
df_gaussian["TIPOVUELO"] = df_gaussian["TIPOVUELO"].str.replace('N','2')
df_gaussian = df_gaussian[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']]
df_gaussian[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']] = df_gaussian[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']].astype(int)
display(df_gaussian.corr())

X = df_gaussian.drop('dummy_atraso',axis=1)
y = df_gaussian['dummy_atraso']
predictors = X
targets = y
X_train, X_test, y_train, y_test  =   train_test_split(predictors, targets, test_size=.8)
print( "Predictor - Training : ", X_train.shape, "Predictor - Testing : ", X_test.shape )

classifier=GaussianNB()
classifier=classifier.fit(X_train,y_train)
predictions=classifier.predict(X_test)
a =  sklearn.metrics.confusion_matrix(y_test,predictions)
print('')
print('')
print('')
print('Falsos correctamente predecidos     = '+str("{:.2f}".format((a[0][0]/(a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')
print('')
print('Verdaderos correctamente predecidos = '+str("{:.2f}".format((a[1][1]/(a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')
print('')
print('Error                               = '+str("{:.2f}".format((a[0][1]+a[1][0])/((a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')

,TIPOVUELO,temporada_alta,dif_min,periodo_dia,dummy_atraso
TIPOVUELO,1.000000,NaN,-0.052451,0.159238,-0.018908
temporada_alta,NaN,NaN,NaN,NaN,NaN
dif_min,-0.052451,NaN,1.000000,-0.019752,0.586589
periodo_dia,0.159238,NaN,-0.019752,1.000000,0.020947
dummy_atraso,-0.018908,NaN,0.586589,0.020947,1.000000


Predictor - Training :  (9088, 4) Predictor - Testing :  (36356, 4)



Falsos correctamente predecidos     = 32.48%

Verdaderos correctamente predecidos = 60.03%

Error                               = 7.50%


#### SVM

In [155]:
df_svm = df_baja.copy()
df_svm.columns
df_svm['dif_min'] = df_svm['dif_min'].astype(int)
lista_atraso = []
for index,row in df_svm.iterrows():
  min = row.get('dif_min')
  if min>=1:
    lista_atraso.append('1')
  else:lista_atraso.append('0')
df_svm['dummy_atraso'] = lista_atraso
df_svm = df_svm.drop(columns=['Fecha-I','Fecha-O','DIA','MES','AÑO','Vlo-I','atraso_15','SIGLAORI','OPERA'])
df_svm["periodo_dia"] = df_svm["periodo_dia"].str.replace('Noche','1')
df_svm["periodo_dia"] = df_svm["periodo_dia"].str.replace('Tarde','2')
df_svm["periodo_dia"] = df_svm["periodo_dia"].str.replace('Mañana','3')

df_svm["TIPOVUELO"] = df_svm["TIPOVUELO"].str.replace('I','1')
df_svm["TIPOVUELO"] = df_svm["TIPOVUELO"].str.replace('N','2')
df_svm = df_svm[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']]
df_svm[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']] = df_svm[['TIPOVUELO','temporada_alta','dif_min','periodo_dia','dummy_atraso']].astype(int)
display(df_svm.corr())

X = df_svm.drop('dummy_atraso',axis=1)
y = df_svm['dummy_atraso']


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.7,random_state=2)

Scaler_X = StandardScaler()
X_train = Scaler_X.fit_transform(X_train)
X_test = Scaler_X.transform(X_test)

rfc = RandomForestClassifier(n_estimators=1)
rfc.fit(X_train,y_train)
rfc__pred = rfc.predict(X_test)
# print(accuracy_score(y_test,rfc__pred))
a = confusion_matrix(y_test,rfc__pred)
# print(classification_report(y_test,rfc__pred))
print('')
print('')
print('')
print('Falsos correctamente predecidos     = '+str("{:.2f}".format((a[0][0]/(a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')
print('')
print('Verdaderos correctamente predecidos = '+str("{:.2f}".format((a[1][1]/(a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')
print('')
print('Error                               = '+str("{:.2f}".format((a[0][1]+a[1][0])/((a[1][1]+a[1][0]+a[0][0]+a[0][1]))*100))+'%')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,TIPOVUELO,temporada_alta,dif_min,periodo_dia,dummy_atraso
TIPOVUELO,1.000000,NaN,-0.052451,0.159238,-0.018908
temporada_alta,NaN,NaN,NaN,NaN,NaN
dif_min,-0.052451,NaN,1.000000,-0.019752,0.586589
periodo_dia,0.159238,NaN,-0.019752,1.000000,0.020947
dummy_atraso,-0.018908,NaN,0.586589,0.020947,1.000000





Falsos correctamente predecidos     = 34.27%

Verdaderos correctamente predecidos = 65.73%

Error                               = 0.00%


#### Redes Neuronales (para predecir los minutos que vamos a tener de atraso)

In [157]:
df_rn = df_baja.copy()
df_rn.columns
df_rn['dif_min'] = df_rn['dif_min'].astype(int)
lista_atraso = []
for index,row in df_rn.iterrows():
  min = row.get('dif_min')
  if min>=1:
    lista_atraso.append('1')
  else:lista_atraso.append('0')
df_rn['dummy_atraso'] = lista_atraso
df_rn = df_rn.drop(columns=['Fecha-O','DIA','MES','AÑO','Vlo-I','atraso_15','SIGLAORI','OPERA'])
df_rn["periodo_dia"] = df_rn["periodo_dia"].str.replace('Noche','1')
df_rn["periodo_dia"] = df_rn["periodo_dia"].str.replace('Tarde','2')
df_rn["periodo_dia"] = df_rn["periodo_dia"].str.replace('Mañana','3')

df_rn["TIPOVUELO"] = df_rn["TIPOVUELO"].str.replace('I','1')
df_rn["TIPOVUELO"] = df_rn["TIPOVUELO"].str.replace('N','2')
df_rn = df_rn[['Fecha-I','dif_min']]
df_rn['dif_min'] = df_rn['dif_min'].astype(int)
df_rn = df_rn[df_rn['dif_min']>0]
df_rn



# # Hacemos la data de prediccion y la que vamos a predecir
atraso1 = df_rn.iloc[15509:]
atraso2 = df_rn.iloc[:15509]

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn import pipeline


PASOS = 20
# convert series to supervised learning
def series_to_supervised(atraso2, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(atraso2) is list else atraso2.shape[1]
    df = pd.DataFrame(atraso2)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# load dataset
values = atraso2["dif_min"].values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, PASOS, 1)

values = reframed.values
n_train_days = 148 - (30+PASOS)
train = values[:n_train_days, :]
test = values[n_train_days:, :]
# split into input and outputs
x_train, y_train = train[:, :-1], train[:, -1]
x_val, y_val = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))



def crear_modeloFF():
    model = Sequential() 
    model.add(Dense(PASOS, input_shape=(1,PASOS),activation='tanh'))
    model.add(Flatten())
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_absolute_error',optimizer='Adam',metrics=["mse"])
    model.summary()
    return model

EPOCHS=40
 
model = crear_modeloFF()
 
history=model.fit(x_train,y_train,epochs=EPOCHS,validation_data=(x_val,y_val),batch_size=PASOS)


ultimosDias = atraso2.tail(30)


values = ultimosDias["dif_min"].values
values = values.astype('float32')
# normalize features
values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, PASOS, 1)
reframed.drop(reframed.columns[[7]], axis=1, inplace=True)
reframed.head()


values = reframed.values
x_test = values[6:, :]
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))


def agregarNuevoValor(x_test,nuevoValor):
    for i in range(x_test.shape[2]-1):
        x_test[0][0][i] = x_test[0][0][i+1]
    x_test[0][0][x_test.shape[2]-1]=nuevoValor
    return x_test
 
results=[]
for i in range(20):
    parcial=model.predict(x_test)
    results.append(parcial[0])
    x_test=agregarNuevoValor(x_test,parcial[0])

adimen = [x for x in results] 
inverted = scaler.inverse_transform(adimen)


prediccion1SemanaDiciembre = pd.DataFrame(inverted)
prediccion1SemanaDiciembre.columns = ['pronostico']
# prediccion1SemanaDiciembre.plot()
print('Error = ' +str("{:.2f}".format((atraso1['dif_min'].sum()-prediccion1SemanaDiciembre['pronostico'].sum())/(atraso1['dif_min'].sum())*100))+'%')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 1, 20)             420       
                                                                 
 flatten_4 (Flatten)         (None, 20)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 21        
                                                                 
Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


5/5 [==============================] - 2s 265ms/step - loss: 0.8686 - mse: 0.9537 - val_loss: 0.8581 - val_mse: 0.8927
Epoch 2/40
5/5 [==============================] - 1s 325ms/step - loss: 0.6969 - mse: 0.6452 - val_loss: 0.6492 - val_mse: 0.5475
Epoch 3/40
5/5 [==============================] - 1s 228ms/step - loss: 0.5565 - mse: 0.4364 - val_loss: 0.4840 - val_mse: 0.3368
Epoch 4/40
5/5 [==============================] - 1s 324ms/step - loss: 0.4566 - mse: 0.3179 - val_loss: 0.3810 - val_mse: 0.2385
Epoch 5/40
5/5 [==============================] - 1s 236ms/step - loss: 0.4071 - mse: 0.2752 - val_loss: 0.3276 - val_mse: 0.2026
Epoch 6/40
5/5 [==============================] - 1s 323ms/step - loss: 0.3826 - mse: 0.2616 - val_loss: 0.3050 - val_mse: 0.1939
Epoch 7/40
5/5 [==============================] - 1s 242ms/step - loss: 0.3699 - mse: 0.2612 - val_loss: 0.2962 - val_mse: 0.1937
Epoch 8/40
5/5 [==============================] - 1s 323ms/step - loss: 0.3638 - mse: 0.2623 - val_lo

Al igual que en la temporada alta, el mejor modelo claramente es el gaussian NB, ya que, este presenta una precision bastante alta y sin overfitting. Producto de esto decidi trabajar con este modelo nomas, de esta forma encontramos una precision muy alta en base a un modelo relativamente simple y de baja capacidad computacional

# Pregunta 5

5.1) ¿Qué performance tiene?

El modelo elegido (Gaussian NB) tiene una precision cercana al 90% y no posee overfitting, lo cual claramente es un buen indicador y nos entrega buenas oportunidades para poder saber que vuelos son los que se van a atrasar y cuales no

5.2) ¿Qué variables son las que más influyen en la predicción?

En este caso se creo una variable dummy, la cual indica que vuelo se atraso y cual no. Esto se creo con el fin de poder clasificar los vuelos que el algoritmo trabajara (ya que, la pregunta va orientada a quienes se van a atrasar, no por cuanto tiempo se van a atrasar).

Esta variable dummy se creo en base a la variable dif_min creada en la pregunta numero 2. Cuando un vuelo se atrasa, tomara el valor de 1, de lo contrario sera 0.

No fue necesario trabajar con mas variables en este caso porque el modelo alcanzo una alta precision con este tipo de variables, por otro lado, no fue necesario porque cubria de forma eficiente lo solicitado para solucionar el problema

5.3) ¿Por qué elegiste ese algoritmo en particular?

Elegi este algoritmo de machine learning porque para el trabajo de clasificacion (y luego prediccion de datos por defecto) es muy poderoso y requiere una baja capacidad computacional para trabajar grandes bases de datos. Este tipo de modelo (y muchas veces tambien el SVM) nos entrega una precision alta trabajando en base a arboles de decision y variables aleatorias, creando de esta forma los caminos mas correctos posibles.

Por otro lado, existia la posibilidad de trabajar con series de tiempo para poder predecir que vuelos se atrasarian y cuales no, lo cual fue descartado porque la estacionalidad no fue significante.

5.4)¿Qué variables son las que más influyen en la predicción?

Considero que las variables que mas influyen son el periodo en el que se hace el vuelo temporada alta o baja) y si el vuelo es nacional o internacional, sin embargo, por temas de simplicidad se trabajo solamente con la primera de estas producto de que el el resultado filtrando solo por esto era lo suficientemente alto como para dejarlo de esta forma 

5.5) ¿Cómo podrías mejorar la performance?

En base al modelo de Gaussian NB considero que no se podria mejorar mucho mas el performane. Sin embargo, considero que existen dos formas de mejorar el performance del modelo:

1) Descargar datos de un periodo mas largo del presentado, de esta forma buscaria indicios de estacionalidad dentro de los datos y trataria de crear un modelo predictivo en base a regresiones lineales o multiples (econometria), o modelos de machine learging, tal como redes neuronales, prophet, Holt Winters, entre otros. Gracias a esto podemos crear un modelo nuevo en base a series temporales y ver si podemos aumentar la precision.

2) Buscar minimizar el overfitting ampliando el entrenamiento con mas datos o con una validacion cruzada. Gracias a esto podemos entrenar el modelo de SVM de mejor manera, o implementar un modelo de SVR.